Encoders

statistical model - cv & tfidf

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import random
random.seed(1234)

In [ ]:
current_p=os.getcwd()
train=pd.read_csv(os.path.join(current_p,'Reddit Data/train.csv'))
valid=pd.read_csv(os.path.join(current_p,'Reddit Data/valid.csv'))
test=pd.read_csv(os.path.join(current_p,'Reddit Data/test.csv'))

In [ ]:
train.head()

,Label_encode,Full_stem
0,1,5 year learn curv
1,0,tank speak upside-down_fac
2,1,sos almost 60 float short guy want fight back ...
3,1,good friday everyon 's week 's haul week 2
4,0,40 sqqq retard hold sqqq right 'm 40 lmfao 'm ...


In [ ]:
class Token(object):
    def __init__(self):
        pass

    def __call__(self,reddit):
        tokens=reddit.split()
        return tokens

In [ ]:
combined_df=pd.concat([train,valid,test],ignore_index=True)

In [ ]:
combined_df.head()

,Label_encode,Full_stem
0,1,5 year learn curv
1,0,tank speak upside-down_fac
2,1,sos almost 60 float short guy want fight back ...
3,1,good friday everyon 's week 's haul week 2
4,0,40 sqqq retard hold sqqq right 'm 40 lmfao 'm ...


In [ ]:
combined_df.shape

(122920, 2)

In [ ]:
combined_df.iloc[88476]

Label_encode          1
Full_stem       rich af
Name: 88476, dtype: object

In [ ]:
train_valid_df=pd.concat([train,valid],ignore_index=True)

In [ ]:
train_valid_df.shape

(110632, 2)

In [ ]:
train_valid_df

,Label_encode,Full_stem
0,1,5 year learn curv
1,0,tank speak upside-down_fac
2,1,sos almost 60 float short guy want fight back ...
3,1,good friday everyon 's week 's haul week 2
4,0,40 sqqq retard hold sqqq right 'm 40 lmfao 'm ...
...,...,...
110627,1,brazil keep give bad boomer
110628,1,avail massag go happi end
110629,1,ai n't much 's honest work
110630,0,let go 's still hold


In [ ]:
train.shape

(88476, 2)

In [ ]:
valid.shape

(22156, 2)

In [ ]:
test.shape

(12288, 2)

In [ ]:
valid.head()

,Label_encode,Full_stem
0,1,rich af
1,1,go silver go amc amc short sever larg fund
2,0,littl lesson hodl took 2x profit januari miss ...
3,1,dvax 15 today beyond
4,1,cciv moon boy rocket


count vector

In [ ]:
count_vectorizer=CountVectorizer(analyzer='word',input='content',
                                 min_df=3, #article frequency, if less than 3 then ignored
                                 ngram_range=(1,1),
                                 tokenizer=Token())#tokenize the text using self-defined processor

In [ ]:
count_vec=count_vectorizer.fit(combined_df['Full_stem'].tolist())

C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
train_idx=list(range(0,train.shape[0]))
valid_idx=list(range(train.shape[0],train_valid_df.shape[0]))

In [ ]:
max(valid_idx)

110631

tf-idf

In [ ]:
tf_idf_vectorizer=TfidfVectorizer(analyzer='word',input='content',
                                  min_df=3, #article frequency, if less than 3 then ignored
                                  ngram_range=(1,1),
                                  tokenizer=Token())#tokenize the text using self-defined processor

In [ ]:
tf_idf_vec=count_vectorizer.fit(combined_df['Full_stem'].tolist())

C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Embeddings

In [ ]:
class SelfVectorizer:
    def __init__(self,model):
        self.word_vectors=model
        self.dim=len(next(iter(model.vectors)))

    def fit(self,data):
        pass

    def transform(self,data):
        #data=> a list of sentences

        #initialize the array
        X = np.zeros((len(data),self.dim))

        for i in range(len(data)):

            sentence=data[i]
            vecs=np.zeros((len(sentence),self.dim))

            for n in range(len(sentence)):
                if sentence[n] in self.word_vectors:
                    vecs[n]=self.word_vectors[sentence[n]]

            X[i]=vecs.mean(axis=0)

        return X

    def fit_transform(self,data):
        self.fit(data)
        return self.transform(data)

Word2vec

In [ ]:
from gensim.models import KeyedVectors

#load word2vec embeddings
model_w2vec=KeyedVectors.load_word2vec_format('./Reddit Data/Word2vec/GoogleNews-vectors-negative300.bin', binary=True)

Glove

In [ ]:
#load glove embeddings
model_glove=KeyedVectors.load_word2vec_format('./Reddit Data/glove/glove.twitter.27B.100d.txt',binary=False,no_header=True)

In [ ]:
word2vec_vectorizer=SelfVectorizer(model_w2vec)
#check_vec=word2_vec.fit_transform(check['Full_stem'].to_list())

In [ ]:
glove_vectorizer=SelfVectorizer(model_glove)
#check_g=g.fit_transform(check['Full_stem'].to_list())

In [ ]:
Doc2Vec

In [ ]:
combined_df['Full_tokens']=combined_df['Full_stem'].apply(lambda x: x.split())

In [ ]:
#https://stackoverflow.com/questions/46786311/how-to-load-the-pre-trained-doc2vec-model-and-use-its-vectors
from gensim.models.doc2vec import TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(combined_df['Full_tokens'].to_list())]

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [ ]:
import multiprocessing
from gensim.models import Doc2Vec
cores = multiprocessing.cpu_count()
model_d2v = Doc2Vec(dm=1, # DM model - distributed memory
                    dm_mean=1, # dm_mean = 1 for using mean of the context word vectors
                    vector_size=100, # no. of desired features
                    window=5,
                    negative=5, # if > 0 then negative sampling will be used
                    min_count=3, # Ignores all words with total frequency lower than 5.
                    workers=cores-3, # no. of cores
                    alpha=0.01, # learning rate
                    seed = 1234, # for reproducibility
                                 )
model_d2v.build_vocab([i for i in tqdm(documents)])

100%|██████████| 122920/122920 [00:00<00:00, 4717522.19it/s]


In [ ]:
model_d2v.train(documents, total_examples= len(combined_df['Full_tokens'].to_list()), epochs=100)

In [ ]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_100dm")
model_d2v.save(fname)

In [ ]:
model_d2v.save_word2vec_format('./Reddit Data/doc2vec/doc2vec_100dm.txt', doctag_vec=True, word_vec=False, prefix='*dt_', fvocab=None, binary=False)

In [ ]:
from gensim.models import KeyedVectors
temp=KeyedVectors.load_word2vec_format('./Reddit Data/doc2vec/doc2vec_100dm.txt')

In [ ]:
temp.vectors

array([[-0.08690852, -0.01198514,  0.02686039, ..., -0.07527874,
         0.06482085,  0.00805522],
       [-0.01778838, -0.03051477,  0.26407024, ...,  0.03051514,
        -0.04122828, -0.02830317],
       [-0.23678134, -0.1692023 ,  0.46942848, ...,  0.46263894,
        -0.60667396, -1.0039922 ],
       ...,
       [ 0.34427056, -0.40881082, -0.21070008, ..., -0.5634346 ,
         0.25433275,  0.09431787],
       [-0.17767964,  0.03863602,  0.35018387, ..., -0.09481397,
         0.10238766, -0.0672965 ],
       [ 0.4797696 , -0.0575099 ,  0.28607655, ...,  0.15998995,
         0.02114091, -0.23809445]], dtype=float32)

In [ ]:
model_d2v.dv.vectors

array([[-0.08690852, -0.01198514,  0.02686039, ..., -0.07527874,
         0.06482085,  0.00805522],
       [-0.01778838, -0.03051477,  0.26407024, ...,  0.03051514,
        -0.04122828, -0.02830317],
       [-0.23678134, -0.1692023 ,  0.46942848, ...,  0.46263894,
        -0.60667396, -1.0039922 ],
       ...,
       [ 0.34427056, -0.40881082, -0.21070008, ..., -0.5634346 ,
         0.25433275,  0.09431787],
       [-0.17767964,  0.03863602,  0.35018387, ..., -0.09481397,
         0.10238766, -0.0672965 ],
       [ 0.4797696 , -0.0575099 ,  0.28607655, ...,  0.15998995,
         0.02114091, -0.23809445]], dtype=float32)

In [ ]:
model_d2v.dv[0]

array([-8.69085193e-02, -1.19851427e-02,  2.68603880e-02,  3.56292188e-01,
        4.76900563e-02,  1.76203430e-01, -7.40177277e-03, -1.12357244e-01,
       -6.67120442e-02,  5.00863828e-02,  5.62294014e-02,  2.12199211e-01,
        7.84320012e-03,  1.34440958e-02, -2.51014620e-01,  1.61867738e-01,
       -1.60084814e-01, -8.77696276e-02, -6.44110367e-02,  2.10547328e-01,
        1.96514666e-01, -1.26974165e-01,  1.27333194e-01, -1.49437249e-01,
       -7.92188197e-02,  2.87601706e-02, -8.94683152e-02, -6.74052015e-02,
       -1.23909600e-01, -1.19323216e-01, -4.65215072e-02, -2.30795130e-01,
        3.72595727e-01, -2.34034002e-01, -5.62383831e-02,  3.13892215e-01,
       -1.71390891e-01, -6.59792051e-02,  1.70361400e-01,  8.18504319e-02,
       -1.83096707e-01,  6.92197755e-02, -1.01427384e-01, -1.13124244e-01,
        1.94041327e-01,  9.87254432e-04,  1.49086937e-01,  1.52714267e-01,
       -1.99541241e-01,  1.75710097e-01,  1.47166222e-01,  3.65142971e-02,
        9.36454907e-02,  

In [ ]:
import numpy as np
docvec_arrays = np.zeros((len(train_valid_df['Full_stem'].tolist()), 100))
for i in range(len(train_valid_df['Full_stem'].tolist())):
    docvec_arrays[i,:] = temp.vectors[i].reshape((1,100))

docvec_df = pd.DataFrame(docvec_arrays)
docvec_df.shape

(110632, 100)

In [ ]:
train_valid_df.shape[0]

110632

In [ ]:
model_d2v.vectors[110632]

array([ 0.41776082, -0.0085654 , -0.15378883, -0.51160216,  0.25980225,
        1.0028585 , -0.32010943,  0.8517619 ,  0.6396408 ,  0.5316336 ,
       -0.17549106,  0.24697304, -0.05236217, -0.75478685, -0.1025875 ,
        0.7953186 ,  1.3241167 ,  0.09251793,  0.31590837,  0.01150908,
       -1.0037675 , -0.90452516,  0.08797783, -0.6850462 , -0.19776446,
       -0.6221488 , -0.01882737,  0.0936541 ,  0.14180717,  0.45509744,
        0.28192922, -0.25961447, -0.09914524, -1.1176361 , -0.4520012 ,
       -0.32127637, -0.10788857, -0.6076995 ,  0.17462085, -0.38395923,
        0.14604065, -0.9815845 ,  0.16310579, -0.47577026,  0.23732843,
       -0.00623911,  0.8840852 ,  0.36110738, -0.2972679 ,  0.2703807 ,
       -0.0275034 , -1.0972515 , -0.40760207, -0.1512076 , -0.264427  ,
       -0.30744183, -0.42974517, -0.78162056,  0.5578623 ,  0.34354323,
       -0.21287648, -0.04845844,  1.0050449 , -0.0305948 ,  0.7492365 ,
       -0.20892408, -0.95996404,  0.11444431,  0.48466834,  0.30

In [ ]:
model_d2v.dv.vectors[110632]

array([ 0.41776082, -0.0085654 , -0.15378883, -0.51160216,  0.25980225,
        1.0028585 , -0.32010943,  0.8517619 ,  0.6396408 ,  0.5316336 ,
       -0.17549106,  0.24697304, -0.05236217, -0.75478685, -0.1025875 ,
        0.7953186 ,  1.3241167 ,  0.09251793,  0.31590837,  0.01150908,
       -1.0037675 , -0.90452516,  0.08797783, -0.6850462 , -0.19776446,
       -0.6221488 , -0.01882737,  0.0936541 ,  0.14180717,  0.45509744,
        0.28192922, -0.25961447, -0.09914524, -1.1176361 , -0.4520012 ,
       -0.32127637, -0.10788857, -0.6076995 ,  0.17462085, -0.38395923,
        0.14604065, -0.9815845 ,  0.16310579, -0.47577026,  0.23732843,
       -0.00623911,  0.8840852 ,  0.36110738, -0.2972679 ,  0.2703807 ,
       -0.0275034 , -1.0972515 , -0.40760207, -0.1512076 , -0.264427  ,
       -0.30744183, -0.42974517, -0.78162056,  0.5578623 ,  0.34354323,
       -0.21287648, -0.04845844,  1.0050449 , -0.0305948 ,  0.7492365 ,
       -0.20892408, -0.95996404,  0.11444431,  0.48466834,  0.30

In [ ]:
docvec_test_arrays

array([[ 0.41776082, -0.0085654 , -0.15378883, ...,  0.39296636,
         0.39276347,  0.3027074 ],
       [-0.08306343, -0.02300319,  0.04958503, ..., -0.0653831 ,
        -0.12780552, -0.05292692],
       [ 0.08124175,  0.14320396, -0.04499749, ..., -0.01676416,
        -0.10418358,  0.04777047],
       ...,
       [ 0.34427056, -0.40881082, -0.21070008, ..., -0.5634346 ,
         0.25433275,  0.09431787],
       [-0.17767964,  0.03863602,  0.35018387, ..., -0.09481397,
         0.10238766, -0.0672965 ],
       [ 0.47976959, -0.0575099 ,  0.28607655, ...,  0.15998995,
         0.02114091, -0.23809445]])

searching for models - tuning

In [ ]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
from sklearn.svm import SVC
svm_rbf=SVC(kernel='rbf',cache_size=10000,
       random_state=1234,max_iter=100000)

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=1234)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
#https://medium.com/grabngoinfo/support-vector-machine-svm-hyperparameter-tuning-in-python-a65586289bcb
#cache_size to a higher value than the default of 200(MB), such as 500(MB) or 1000(MB).
#scores={'f1':make_scorer(f1_score),'mcc':make_scorer(matthews_corrcoef)}
param_svm = { 'C':[0.1, 1, 10, 50, 100, 200, 1000]}
#grid = GridSearchCV(SVC(),param_grid,scoring=scores,refit='mcc')
search_svm = RandomizedSearchCV(svm_rbf,
                          param_distributions=param_svm,
                          n_iter=7, cv=((train_idx, valid_idx),),
                          verbose=1, n_jobs=20,
                          return_train_score=True)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
params_xgb = {
    "colsample_bytree": stats.uniform(0.7, 0.3),
    "gamma": stats.uniform(0, 0.5),
    "learning_rate": stats.uniform(0.03, 0.3), # default 0.1
    "max_depth": stats.randint(3, 10), # default 3
    "n_estimators": stats.randint(100, 150), # default 100
    "subsample": stats.uniform(0.6, 0.4)
}

search_xgb = RandomizedSearchCV(xgb_model,
                            param_distributions=params_xgb,
                            n_iter=100,
                            cv=((train_idx, valid_idx),),
                            verbose=1, n_jobs=20,
                            return_train_score=True)

In [ ]:
y_train_valid=np.asarray(train_valid_df['Label_encode'].tolist())

encoder={'cv':count_vec,
         'tf-idf':tf_idf_vec,
         'word2vec':word2vec_vectorizer,
         'glove':glove_vectorizer,
         'doc2vec':docvec_arrays
}

best_params=dict()

for k,v in encoder.items():
    print(f'Encoder: {k}')
    if k!='doc2vec':
        best_params[k]=dict()
        X_train_valid=v.transform(train_valid_df['Full_stem'].tolist())
        print('Model: SVM')
        search_svm.fit(X_train_valid,y_train_valid)
        report_best_scores(search_svm.cv_results_,1)
        best_params[k]['SVM']=search_svm.best_params_
        print('Model: XGB')
        search_xgb.fit(X_train_valid,y_train_valid)
        report_best_scores(search_xgb.cv_results_,1)
        best_params[k]['XGB']=search_xgb.best_params_

    elif k=='doc2vec':
        best_params[k]=dict()
        print('Model: SVM')
        search_svm.fit(v,y_train_valid)
        report_best_scores(search_svm.cv_results_,1)
        best_params[k]['SVM']=search_svm.best_params_
        print('Model: XGB')
        search_xgb.fit(v,y_train_valid)
        report_best_scores(search_xgb.cv_results_,1)
        best_params[k]['XGB']=search_xgb.best_params_

    else:
        raise KeyError

Encoder: cv
Model: SVM
Fitting 1 folds for each of 7 candidates, totalling 7 fits


C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Model with rank: 1
Mean validation score: 0.627 (std: 0.000)
Parameters: {'C': 10}

Model: XGB
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Model with rank: 1
Mean validation score: 0.751 (std: 0.000)
Parameters: {'colsample_bytree': 0.7252613025302761, 'gamma': 0.06120957109133085, 'learning_rate': 0.29475817611426813, 'max_depth': 8, 'n_estimators': 147, 'subsample': 0.8724517161347709}

Encoder: tf-idf
Model: SVM
Fitting 1 folds for each of 7 candidates, totalling 7 fits


C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Model with rank: 1
Mean validation score: 0.627 (std: 0.000)
Parameters: {'C': 10}

Model: XGB
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Model with rank: 1
Mean validation score: 0.751 (std: 0.000)
Parameters: {'colsample_bytree': 0.7687925557433345, 'gamma': 0.39696604182138123, 'learning_rate': 0.2728618017864384, 'max_depth': 9, 'n_estimators': 139, 'subsample': 0.8827847759051652}

Encoder: word2vec
Model: SVM
Fitting 1 folds for each of 7 candidates, totalling 7 fits


...

Model with rank: 1
Mean validation score: 0.513 (std: 0.000)
Parameters: {'C': 100}

Model: XGB
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Model with rank: 1
Mean validation score: 0.653 (std: 0.000)
Parameters: {'colsample_bytree': 0.7864969729699637, 'gamma': 0.4128876995773074, 'learning_rate': 0.03300573518913107, 'max_depth': 8, 'n_estimators': 141, 'subsample': 0.9326435765031607}

Encoder: glove
Model: SVM
Fitting 1 folds for each of 7 candidates, totalling 7 fits


C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Model with rank: 1
Mean validation score: 0.499 (std: 0.000)
Parameters: {'C': 200}

Model: XGB
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Model with rank: 1
Mean validation score: 0.651 (std: 0.000)
Parameters: {'colsample_bytree': 0.7062342760855846, 'gamma': 0.22710723797323829, 'learning_rate': 0.09451568476798504, 'max_depth': 9, 'n_estimators': 126, 'subsample': 0.9220052319836918}

Encoder: doc2vec
Model: SVM
Fitting 1 folds for each of 7 candidates, totalling 7 fits


C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Model with rank: 1
Mean validation score: 0.572 (std: 0.000)
Parameters: {'C': 1000}

Model: XGB
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Model with rank: 1
Mean validation score: 0.704 (std: 0.000)
Parameters: {'colsample_bytree': 0.9086493002188916, 'gamma': 0.10493109942261192, 'learning_rate': 0.1476369073001081, 'max_depth': 7, 'n_estimators': 145, 'subsample': 0.9189417586422284}



In [ ]:
encoder={'cv':count_vec,
         'tf-idf':tf_idf_vec,
         'word2vec':word2vec_vectorizer,
         'glove':glove_vectorizer,
         'doc2vec':docvec_arrays
}

In [ ]:
for k,v in encoder.items():
    print(f'Encoder: {k}')
    if k=='doc2vec':
        best_params[k]=dict()
        print('Model: SVM')
        search_svm.fit(v,y_train_valid)
        report_best_scores(search_svm.cv_results_,1)
        best_params[k]['SVM']=search_svm.best_params_

Encoder: doc2vec
Model: XGB
Fitting 1 folds for each of 100 candidates, totalling 100 fits
Model with rank: 1
Mean validation score: 0.703 (std: 0.000)
Parameters: {'colsample_bytree': 0.8992496480396803, 'gamma': 0.2673403210953481, 'learning_rate': 0.04087451813566916, 'max_depth': 9, 'n_estimators': 141, 'subsample': 0.7849498578404374}



In [ ]:
with open('./Reddit Data/model/best_param.txt','w') as f:
    for k,v in best_params.items():
        text=k+'\n'+str(v)+'\n'
        f.write(text)
    f.close()

In [ ]:
best_params=dict()
with open('./Reddit Data/model/best_param.txt','r') as f:
    text=f.readlines()
    text=[each.strip('\n') for each in text]
    for i in range(len(text)):
        if i%2==0:
            best_params[text[i]]=eval(text[i+1])
    f.close()

In [ ]:
from sklearn.metrics import matthews_corrcoef,accuracy_score,f1_score,precision_score,recall_score
def get_evaluation_metrics(y_true,y_pred):
    metrics_dict=dict()
    metrics_dict['accuracy']=accuracy_score(y_true,y_pred)
    metrics_dict['precision']=precision_score(y_true,y_pred)
    metrics_dict['recall']=recall_score(y_true,y_pred)
    metrics_dict['f1']=f1_score(y_true,y_pred)
    metrics_dict['matthews_corr']=matthews_corrcoef(y_true,y_pred)
    return metrics_dict

In [ ]:
best_params

{'doc2vec': {'SVM': {'C': 1},
  'XGB': {'colsample_bytree': 0.8992496480396803,
   'gamma': 0.2673403210953481,
   'learning_rate': 0.04087451813566916,
   'max_depth': 9,
   'n_estimators': 141,
   'subsample': 0.7849498578404374}},
 'cv': {'SVM': {'C': 10},
  'XGB': {'colsample_bytree': 0.7252613025302761,
   'gamma': 0.06120957109133085,
   'learning_rate': 0.29475817611426813,
   'max_depth': 8,
   'n_estimators': 147,
   'subsample': 0.8724517161347709}},
 'tf_idf': {'SVM': {'C': 10},
  'XGB': {'colsample_bytree': 0.7687925557433345,
   'gamma': 0.39696604182138123,
   'learning_rate': 0.2728618017864384,
   'max_depth': 9,
   'n_estimators': 139,
   'subsample': 0.8827847759051652}},
 'word2vec': {'SVM': {'C': 100},
  'XGB': {'colsample_bytree': 0.7864969729699637,
   'gamma': 0.4128876995773074,
   'learning_rate': 0.03300573518913107,
   'max_depth': 8,
   'n_estimators': 141,
   'subsample': 0.9326435765031607}},
 'glove': {'SVM': {'C': 200},
  'XGB': {'colsample_bytree': 0.70

In [ ]:
import skops.io as sio
from sklearn.metrics import roc_curve, auc
def train_test_evaluation(model,X_train,y_train,X_test,y_test,fname):
    #train the model
    model.fit(X_train,y_train)
    #predict
    y_pred=model.predict(X_test)
    #y_prob=model.predict_proba(X_test)[:,1]
    #evaluates
    merics=get_evaluation_metrics(y_test,y_pred)
    #fpr, tpr, threshold = roc_curve(y_test, y_prob)
    #roc_auc = auc(fpr, tpr)
    #merics['AUC']=roc_auc
    #save the model
    sio.dump(model,fname)
    #return the performance metrics
    return y_pred,merics

In [ ]:
import xgboost as xgb
y_train=np.asarray(train['Label_encode'].tolist())
y_test=np.asarray(test['Label_encode'].tolist())
model_eval_dict=dict()

for k,v in best_params.items():
    if k!='doc2vec':
        X_train=encoder[k].transform(train['Full_stem'].tolist())
        X_test=encoder[k].transform(test['Full_stem'].tolist())
    elif k=='doc2vec':
        X_train=docvec_train_arrays
        X_test=docvec_test_arrays
    else:
        raise KeyError

    svm=SVC(kernel='rbf',cache_size=10000,random_state=1234,max_iter=100000,**v['SVM'],probability=True)
    fname_svm=f'./Reddit Data/model/{k}_svm.skops'
    y_pred_svm,y_prob_svm,evaluation_svm=train_test_evaluation(svm,X_train,y_train,X_test,y_test,fname_svm)
    model_eval_dict[f'{k}+SVM']=[y_pred_svm,y_prob_svm,y_test,evaluation_svm]
    print(f'{k}+SVM')
    print(f'parameters: {v["SVM"]}')
    print(evaluation_svm)

    fname_xgb=f'./Reddit Data/model/{k}_xgb.skops'
    xgbm= xgb.XGBClassifier(objective="binary:logistic", random_state=1234,**v['XGB'])
    y_pred_xgb,y_prob_xgb,evaluation_xgb=train_test_evaluation(xgbm,X_train,y_train,X_test,y_test,fname_xgb)
    model_eval_dict[f'{k}+XGB']=[y_pred_xgb,y_prob_xgb,y_test,evaluation_xgb]
    print(f'{k}+XGB')
    print(f'parameters: {v["XGB"]}')
    print(evaluation_xgb)

In [ ]:
import xgboost as xgb
model_eval_svm=dict()

for k,v in best_params.items():
    if k=='doc2vec':
        X_train=docvec_train_arrays
        X_test=docvec_test_arrays

    svm=SVC(kernel='rbf',cache_size=10000,random_state=1234,max_iter=100000,**v['SVM'])
    fname_svm=f'./Reddit Data/model/{k}_svmV2.skops'
    y_pred_svm,evaluation_svm=train_test_evaluation(svm,X_train,y_train,X_test,y_test,fname_svm)
    model_eval_dict[f'{k}+SVM']=[y_pred_svm,evaluation_svm]
    print(f'{k}+SVM')
    print(evaluation_svm)

    fname_xgb=f'./Reddit Data/model/{k}_xgb.skops'
    xgbm= xgb.XGBClassifier(objective="binary:logistic", random_state=1234,**v['XGB'])
    y_pred_xgb,y_prob_xgb,evaluation_xgb=train_test_evaluation(xgbm,X_train,y_train,X_test,y_test,fname_xgb)
    model_eval_dict[f'{k}+XGB']=[y_pred_xgb,y_prob_xgb,y_test,evaluation_xgb]
    print(f'{k}+XGB')
    print(f'parameters: {v["XGB"]}')
    print(evaluation_xgb)

C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


cv+SVM
{'accuracy': 0.763671875, 'precision': 0.7571283095723014, 'recall': 0.7517694641051568, 'f1': 0.7544393708777271, 'matthews_corr': 0.5266925432899674}


C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


tf-idf+SVM
{'accuracy': 0.763671875, 'precision': 0.7571283095723014, 'recall': 0.7517694641051568, 'f1': 0.7544393708777271, 'matthews_corr': 0.5266925432899674}


C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


word2vec+SVM
{'accuracy': 0.618896484375, 'precision': 0.5898577790547335, 'recall': 0.6919447253117628, 'f1': 0.6368359829391237, 'matthews_corr': 0.24465237190663286}


C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


glove+SVM
{'accuracy': 0.6229654947916666, 'precision': 0.6051058329293909, 'recall': 0.6311088641725648, 'f1': 0.6178338695042482, 'matthews_corr': 0.24633186323137807}


C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


doc2vec+SVM
{'accuracy': 0.5582682291666666, 'precision': 0.5471487141259784, 'recall': 0.4947758678800135, 'f1': 0.5196460176991151, 'matthews_corr': 0.11318507053350044}


In [ ]:
docvec_train_arrays

array([[-0.08690852, -0.01198514,  0.02686039, ..., -0.07527874,
         0.06482085,  0.00805522],
       [-0.01778838, -0.03051477,  0.26407024, ...,  0.03051514,
        -0.04122828, -0.02830317],
       [-0.23678134, -0.1692023 ,  0.46942848, ...,  0.46263894,
        -0.60667396, -1.0039922 ],
       ...,
       [-0.4940564 , -0.13408628,  0.32272193, ..., -0.24110205,
        -0.08822834, -0.04427361],
       [-0.06563839,  0.00643401,  0.01659452, ..., -0.07860819,
         0.0113459 , -0.05335868],
       [ 0.05549354,  0.18212068, -0.03450234, ...,  0.04791186,
         0.27553603, -0.32263654]])

In [ ]:
import xgboost as xgb
from sklearn.svm import SVC
model_eval_dict=dict()

y_train=np.asarray(train['Label_encode'].tolist())
y_test=np.asarray(test['Label_encode'].tolist())

for k,v in best_params.items():
    if k=='doc2vec':
        X_train=docvec_train_arrays
        X_test=docvec_test_arrays
    else:
        continue

    #svm=SVC(kernel='rbf',cache_size=10000,random_state=1234,max_iter=100000,**v['SVM'])
    fname_svm=f'./Reddit Data/model/{k}_svmV2.skops'
    #y_pred_svm,evaluation_svm=train_test_evaluation(svm,X_train,y_train,X_test,y_test,fname_svm)
    model_eval_dict[f'{k}+SVM']=[y_pred_svm,evaluation_svm]
    print(f'{k}+SVM')
    print(evaluation_svm)

    fname_xgb=f'./Reddit Data/model/{k}_xgb.skops'
    xgbm= xgb.XGBClassifier(objective="binary:logistic", random_state=1234,**v['XGB'])
    y_pred_xgb,evaluation_xgb=train_test_evaluation(xgbm,X_train,y_train,X_test,y_test,fname_xgb)
    model_eval_dict[f'{k}+XGB']=[y_pred_xgb,evaluation_xgb]
    print(f'{k}+XGB')
    print(f'parameters: {v["XGB"]}')
    print(evaluation_xgb)

doc2vec+SVM
{'accuracy': 0.721435546875, 'precision': 0.7392795883361921, 'recall': 0.653690596562184, 'f1': 0.6938556479742419, 'matthews_corr': 0.44288240592934086}
doc2vec+XGB
parameters: {'colsample_bytree': 0.8992496480396803, 'gamma': 0.2673403210953481, 'learning_rate': 0.04087451813566916, 'max_depth': 9, 'n_estimators': 141, 'subsample': 0.7849498578404374}
{'accuracy': 0.705322265625, 'precision': 0.7131011608623549, 'recall': 0.6521739130434783, 'f1': 0.6812780565091102, 'matthews_corr': 0.40969284170361514}


In [ ]:
model_eval_dict

{'doc2vec+SVM': [array([0, 0, 1, ..., 1, 0, 1]),
  {'accuracy': 0.721435546875,
   'precision': 0.7392795883361921,
   'recall': 0.653690596562184,
   'f1': 0.6938556479742419,
   'matthews_corr': 0.44288240592934086}],
 'doc2vec+XGB': [array([0, 0, 1, ..., 1, 0, 1]),
  {'accuracy': 0.705322265625,
   'precision': 0.7131011608623549,
   'recall': 0.6521739130434783,
   'f1': 0.6812780565091102,
   'matthews_corr': 0.40969284170361514}]}

In [ ]:
output_list=[]
for k,v in model_eval_dict.items():
    a_tuple=(k,v[1]['accuracy'],v[1]['precision'],v[1]['recall'],v[1]['f1'],v[1]['matthews_corr'],list(v[0]),list(y_test))
    output_list.append(a_tuple)

output_pd=pd.DataFrame.from_records(output_list,columns=['encode_algo','accuracy','precision','recall','f1','matthews_corr','y_pred','y_true'])

In [ ]:
output_pd

,encode_algo,accuracy,precision,recall,f1,matthews_corr,y_pred,y_true
0,doc2vec+SVM,0.721436,0.739280,0.653691,0.693856,0.442882,"[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
1,doc2vec+XGB,0.705322,0.713101,0.652174,0.681278,0.409693,"[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."


In [ ]:
output_pd.to_csv('./Reddit Data/evaluations_doc2vec.csv',index=False)

tf-idf & count vector

In [ ]:
output_list=[]
for k,v in model_eval_dict.items():
    a_tuple=(k,v[1]['accuracy'],v[1]['precision'],v[1]['recall'],v[1]['f1'],v[1]['matthews_corr'],list(v[0]),list(y_test))
    output_list.append(a_tuple)

output_pd=pd.DataFrame.from_records(output_list,columns=['encode_algo','accuracy','precision','recall','f1','matthews_corr','y_pred','y_true'])

In [ ]:
output_pd

,encode_algo,accuracy,precision,recall,f1,matthews_corr,y_pred,y_true
0,cv+SVM,0.763672,0.757128,0.751769,0.754439,0.526693,"[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
1,cv+XGB,0.748861,0.758721,0.703741,0.730198,0.497154,"[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
2,tf-idf+SVM,0.763672,0.757128,0.751769,0.754439,0.526693,"[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
3,tf-idf+XGB,0.750407,0.763366,0.700202,0.730421,0.500473,"[0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
4,word2vec+SVM,0.618896,0.589858,0.691945,0.636836,0.244652,"[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
5,word2vec+XGB,0.646240,0.645516,0.593192,0.618249,0.290649,"[0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
6,glove+SVM,0.622965,0.605106,0.631109,0.617834,0.246332,"[1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
7,glove+XGB,0.652344,0.650543,0.605157,0.627030,0.302959,"[1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
8,doc2vec+SVM,0.558268,0.547149,0.494776,0.519646,0.113185,"[0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
9,doc2vec+XGB,0.698242,0.701850,0.652174,0.676101,0.395293,"[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."


In [ ]:
output_pd.to_csv('./Reddit Data/evaluations_ml.csv',index=False)

In [ ]:
a_df=pd.read_csv('./Reddit Data/evaluations_ml.csv')
a_df

,encode_algo,accuracy,precision,recall,f1,matthews_corr,y_pred,y_true
0,cv+SVM,0.763672,0.757128,0.751769,0.754439,0.526693,"[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
1,cv+XGB,0.748861,0.758721,0.703741,0.730198,0.497154,"[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
2,tf-idf+SVM,0.763672,0.757128,0.751769,0.754439,0.526693,"[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
3,tf-idf+XGB,0.750407,0.763366,0.700202,0.730421,0.500473,"[0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
4,word2vec+SVM,0.618896,0.589858,0.691945,0.636836,0.244652,"[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
5,word2vec+XGB,0.646240,0.645516,0.593192,0.618249,0.290649,"[0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
6,glove+SVM,0.622965,0.605106,0.631109,0.617834,0.246332,"[1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
7,glove+XGB,0.652344,0.650543,0.605157,0.627030,0.302959,"[1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
8,doc2vec+SVM,0.558268,0.547149,0.494776,0.519646,0.113185,"[0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
9,doc2vec+XGB,0.698242,0.701850,0.652174,0.676101,0.395293,"[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."


Word2Vec
https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/
https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial


In [ ]:
import multiprocessing
from gensim.models import Word2Vec

In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
#sg=>1 skip gram
## 0 for CBOW, 1 for skip-gram
w2v_model = Word2Vec(min_count=3,
                     window=2,
                     vector_size=100,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=cores-1)

In [ ]:
corpus=[row.split() for row in combined_df.Full_stem]

In [ ]:
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from time import time

In [ ]:
t = time()
w2v_model.build_vocab(corpus, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 01:03:15: collecting all words and their counts
INFO - 01:03:15: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 01:03:15: PROGRESS: at sentence #10000, processed 91226 words, keeping 9896 word types
INFO - 01:03:15: PROGRESS: at sentence #20000, processed 182525 words, keeping 15006 word types
INFO - 01:03:15: PROGRESS: at sentence #30000, processed 274445 words, keeping 18993 word types
INFO - 01:03:15: PROGRESS: at sentence #40000, processed 366956 words, keeping 22453 word types
INFO - 01:03:15: PROGRESS: at sentence #50000, processed 462009 words, keeping 25584 word types
INFO - 01:03:15: PROGRESS: at sentence #60000, processed 556055 words, keeping 28603 word types
INFO - 01:03:15: PROGRESS: at sentence #70000, processed 647620 words, keeping 31400 word types
INFO - 01:03:15: PROGRESS: at sentence #80000, processed 740100 words, keeping 33944 word types
INFO - 01:03:15: PROGRESS: at sentence #90000, processed 832064 words, keeping 36165 word types


Time to build vocab: 0.01 mins


In [ ]:
t = time()

w2v_model.train(corpus, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 01:03:31: Word2Vec lifecycle event {'msg': 'training model with 23 workers on 13408 vocabulary and 100 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2 shrink_windows=True', 'datetime': '2023-02-24T01:03:31.322060', 'gensim': '4.1.2', 'python': '3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'train'}
INFO - 01:03:31: worker thread finished; awaiting finish of 22 more threads
INFO - 01:03:31: worker thread finished; awaiting finish of 21 more threads
INFO - 01:03:31: worker thread finished; awaiting finish of 20 more threads
INFO - 01:03:31: worker thread finished; awaiting finish of 19 more threads
INFO - 01:03:31: worker thread finished; awaiting finish of 18 more threads
INFO - 01:03:31: worker thread finished; awaiting finish of 17 more threads
INFO - 01:03:31: worker thread finished; awaiting finish of 16 more threads
INFO - 01:03:31: worker thread finished; awaiting finish of 15 m

Time to train the model: 0.86 mins


In [ ]:
#As we do not plan to train the model any further, we are calling init_sims(), which will make the model much more memory-efficient:
w2v_model.init_sims(replace=True)

C:\Users\Tracy\AppData\Local\Temp/ipykernel_14728/2616255580.py:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
WARNING - 01:04:36: destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


In [ ]:
test_df['Full_tokens']=test_df['Full_cleaned'].apply(lambda x: word_tokenize(x))

C:\Users\Tracy\AppData\Local\Temp/ipykernel_16680/997041463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Full_tokens']=test_df['Full_cleaned'].apply(lambda x: word_tokenize(x))


In [ ]:
#http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
class MeanEmbeddingVectorizer(object):
    def __init__(self, model):
        #https://github.com/vijayaiitk/NLP-text-classification-model/issues/1
        self.word2vec = dict(zip(model.wv.index_to_key, model.wv.vectors))
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(self.word2vec.values())))

    def fit_transform(self, x):
        return np.asarray([np.mean([self.word2vec[w] if w in self.word2vec else np.zeros(self.dim) for w in words], axis=0) for words in x])

In [ ]:
train.Full_stem

In [ ]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2v_model)
mean_embedded = mean_embedding_vectorizer.fit_transform(train['Full_stem'].tolist())

In [ ]:
mean_embedded_test = mean_embedding_vectorizer.fit_transform(test['Full_stem'].tolist())
mean_embedded_train_valid = mean_embedding_vectorizer.fit_transform(train_valid_df['Full_stem'].tolist())

In [ ]:
best_params['word2vec']['s']

{'SVM': {'C': 100},
 'XGB': {'colsample_bytree': 0.7864969729699637,
  'gamma': 0.4128876995773074,
  'learning_rate': 0.03300573518913107,
  'max_depth': 8,
  'n_estimators': 141,
  'subsample': 0.9326435765031607}}

In [ ]:
svm=SVC(kernel='rbf',cache_size=10000,random_state=1234,max_iter=100000,**best_params['word2vec']['SVM'])

In [ ]:
svm.fit(mean_embedded,y_train)

C:\Users\Tracy\PycharmProjects\FIT9136\venv\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(C=100, cache_size=10000, max_iter=100000, random_state=1234)

In [ ]:
y_pred=svm.predict(mean_embedded_test)
merics=get_evaluation_metrics(y_test,y_pred)
merics

{'accuracy': 0.5052083333333334,
 'precision': 0.49069589599796076,
 'recall': 0.6488035052241321,
 'f1': 0.5587808417997097,
 'matthews_corr': 0.02070706676642968}

In [ ]:
xgbm= xgb.XGBClassifier(objective="binary:logistic", random_state=1234,**best_params['word2vec']['XGB'])

In [ ]:
xgbm.fit(mean_embedded,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7864969729699637, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.4128876995773074, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03300573518913107, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=141, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1234, ...)

In [ ]:
y_pred=xgbm.predict(mean_embedded_test)
merics=get_evaluation_metrics(y_test,y_pred)
merics

{'accuracy': 0.6272786458333334,
 'precision': 0.6338473705021748,
 'recall': 0.5402763734411864,
 'f1': 0.5833333333333334,
 'matthews_corr': 0.25263901565846225}